In [ ]:
import cv2
import numpy as np
import os

def extract_script(image_path, output_dir):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Load the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Step 1: Preprocessing - Denoising and Thresholding
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    binary_image = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2
    )

    # Step 2: Morphological Operations
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    morphed = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Step 3: Edge Detection
    edges = cv2.Canny(morphed, 50, 150)

    # Step 4: Find Contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Step 5: Initialize an output image and list to store extracted regions
    output_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    extracted_regions = []

    for i, contour in enumerate(contours):
        x, y, w, h = cv2.boundingRect(contour)

        # Filter out small or irrelevant boxes based on size thresholds
        if 20 < w < 200 and 20 < h < 200:  # Adjust these thresholds as needed
            # Draw the bounding box
            cv2.rectangle(output_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Extract the region of interest
            roi = image[y:y+h, x:x+w]
            extracted_regions.append(roi)

            # Save each region as a separate image
            roi_path = os.path.join(output_dir, f"extracted_region_{i}.png")
            cv2.imwrite(roi_path, roi)

    # Save the output image with bounding boxes
    output_image_path = os.path.join(output_dir, "output_with_bounding_boxes.png")
    cv2.imwrite(output_image_path, output_image)

    return output_image_path, len(extracted_regions)

# Define file paths
image_path = '/Users/akshaykumar/Documents/Projects/OCR-practise/edge/tululipi.png'
output_dir = 'edge/extracted_regions2'

# Extract script and save results
output_image_path, num_regions = extract_script(image_path, output_dir)

output_image_path, num_regions


In [1]:
import cv2
import os

def extract_script_with_edges(image_path, output_dir):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image not found or unable to read: {image_path}")

    # Step 1: Convert to Grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Step 2: Preprocessing - Denoising and Thresholding
    blurred = cv2.GaussianBlur(gray_image, (5, 5), 0)
    binary_image = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2
    )

    # Step 3: Edge Detection
    edges = cv2.Canny(binary_image, 50, 150)

    # Step 4: Morphological Operations to improve contours
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    morphed_edges = cv2.dilate(edges, kernel, iterations=1)

    # Step 5: Find Contours for Bounding Boxes
    contours, _ = cv2.findContours(morphed_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Step 6: Draw Bounding Boxes and Save Extracted Regions
    output_image = cv2.cvtColor(gray_image, cv2.COLOR_GRAY2BGR)
    extracted_regions = []

    for i, contour in enumerate(contours):
        x, y, w, h = cv2.boundingRect(contour)
        if 20 < w < 200 and 20 < h < 200:  # Adjust size thresholds as needed
            cv2.rectangle(output_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            roi = gray_image[y:y+h, x:x+w]
            extracted_regions.append(roi)
            roi_path = os.path.join(output_dir, f"extracted_region_{i}.png")
            cv2.imwrite(roi_path, roi)

    # Save the output image with bounding boxes
    output_image_path = os.path.join(output_dir, "output_with_bounding_boxes.png")
    cv2.imwrite(output_image_path, output_image)

    return output_image_path, len(extracted_regions)

# Example usage
image_path = '/Users/akshaykumar/Documents/Projects/OCR-practise/edge/Tulu_glyphs.png'  # Update with your PNG file path
output_dir = '/Users/akshaykumar/Documents/Projects/OCR-practise/edge/edge2/one'  # Update with your desired output directory

# Run the script
output_image_path, num_regions = extract_script_with_edges(image_path, output_dir)
print(f"Output image saved at: {output_image_path}")
print(f"Number of regions extracted: {num_regions}")


Output image saved at: /Users/akshaykumar/Documents/Projects/OCR-practise/edge/edge2/one/output_with_bounding_boxes.png
Number of regions extracted: 454
